# Warm-up examples

In [ ]:
%matplotlib inline
from stimator import read_model
from ipywidgets import interact
import seaborn as sns

from classhelper import tc2df

styles = ['seaborn-whitegrid', 'seaborn-poster']

## Closed system, reversible Michaelis-Menten enzyme

<span style="font-size:150%;">A ⇄ P</span>, catalyzed by an enzyme with kinetics $v = (V \cdot (A - \frac{P}{K_{eq}}))\:/\: (1 + \frac{A}{K_{mA}} + \frac{P}{K_{mP}})$

![](images/cicr.png)

In [ ]:
model = read_model("""
title Closed system, reversible Michaelis-Menten enzyme

v1: A -> P, (V * (A - P / Keq)) / (1 + A/KmA + P/KmP)

V = 1
Keq = 2

KmA = 2
KmP = 1

init: A = 12, P = 0

""")

In [ ]:
result = model.solve(tf=40)
tc2df(result)

In [ ]:
model.solve(tf=40).plot(fig_size=(12,8), style=styles)

### Changing parameters

- the initial value of A 
- the value of KmA

In [ ]:
@interact(A=(6, 12, 0.5), KmA=(1, 5, 0.1))
def change_A(A=12, KmA=2):
    model.init.A = A
    model.parameters.KmA = KmA
    model.solve(tf=40).plot(fig_size=(12,8), yrange=(0,12), style=styles)

## Closed system, several reversible Michaelis-Menten enzymes

![](closed_multi.png)

In [ ]:
model = read_model("""
title Linear pathway

v1: A -> B, (V * (A - B / Keq)) / (1 + A/KmA + B/KmB), V = 1, Keq = 2, KmA = 2, KmB = 1
v2: B -> C, (V * (B - C / Keq)) / (1 + B/KmB + C/KmC), V = 0.5, Keq = 2, KmB = 2, KmC = 1
v3: C -> D, (V * (C - D / Keq)) / (1 + C/KmC + D/KmD), V = 1, Keq = 2, KmC = 2, KmD = 1

init: A = 12

""")

In [ ]:
model.solve(tf=150).plot(fig_size=(12,8), style=styles)

In [ ]:
steady_state = model.solve(tf=2000).last

steady_state

In [ ]:
A, B, C, D = steady_state['A'], steady_state['B'], steady_state['C'], steady_state['D']

r1 = B/A
r2 = C/B
r3 = D/C

print(f'B/A = {r1:.2f}')
print(f'C/B = {r2:.2f}')
print(f'D/C = {r3:.2f}')

## Open system, several reversible Michaelis-Menten enzymes (metabolic pathway)

![](open_linear.png)

In [ ]:
model = read_model("""
title Linear pathway

v1: A -> B, (V * (A - B / Keq)) / (1 + A/KmA + B/KmB), V = 1, Keq = 2, KmA = 2, KmB = 1
v2: B -> C, (V * (B - C / Keq)) / (1 + B/KmB + C/KmC), V = 0.5, Keq = 2, KmB = 2, KmC = 1
v3: C -> D, (V * (C - D / Keq)) / (1 + C/KmC + D/KmD), V = 1, Keq = 2, KmC = 2, KmD = 1

vAin  : -> A, kin, kin = 1
vAout : A ->, koutA * A, koutA = 1

vout  : D ->, kout * D, kout = 2
""")

In [ ]:
model.solve(tf=20).plot(fig_size=(12,8), style=styles)

<span style="font-size:150%;">&ReverseEquilibrium; A ⇄ B ⇄ C ⇄ D &rarr;</span>, computing the values at **steady-state**

In [ ]:
steady_state = model.solve(tf=2000).last

steady_state

In [ ]:
A, B, C, D = steady_state['A'], steady_state['B'], steady_state['C'], steady_state['D']

r1 = B/A
r2 = C/B
r3 = D/C

print(f'B/A = {r1:.2f}')
print(f'C/B = {r2:.2f}')
print(f'D/C = {r3:.2f}')

<span style="font-size:150%;">&ReverseEquilibrium; A ⇄ B ⇄ C ⇄ D &rarr;</span>, computing the rates of reactions at **steady-state**

In [ ]:
steady_state = model.solve(tf=20, outputs='>>').plot(fig_size=(12,8), style=styles)

#### Changing the parameter $kin$ (rate of input of A into the pathway)

In [ ]:
@interact(kin=(1, 5, 0.2))
def change_kin(kin=1):
    model.parameters.vAin.kin = kin
    model.solve(tf=20).plot(fig_size=(12,8), yrange=(0,4), style=styles)

#### Changing the parameter $kin$ (rate of input of A into the pathway), but displying the rates of reactions

In [ ]:
@interact(kin=(1, 5, 0.2))
def change_kin(kin=1):
    model.parameters.vAin.kin = kin
    model.solve(tf=20, outputs='v1 v2 v3'.split()).plot(fig_size=(12,8), yrange=(0,1), style=styles)